# 1. Get training edges

In [ ]:
import networkx as nx
import csv
import random

In [ ]:
G = nx.DiGraph()
fh = open("train.txt", "r")
L = fh.readlines()

In [ ]:
#Feed edges into the graph except those with empty collection of sinks
for line in L:
    lst_edges = []
    for x in line.split('\t'):
        if x != '':
            lst_edges.append(int(x))
    if len(lst_edges) > 1:
        lst_edges_to_add = [(lst_edges[0],e) for e in lst_edges[1:]]
        G.add_edges_from(lst_edges_to_add)
len(G.nodes), len(G.edges)

In [ ]:
# Create arbitrary amount of valid pairs
def create_real_edges(num_obs, graph_edges):
    true_edges = random.choices(graph_edges, k=num_obs)
    true_edges = [[source, sink] for source, sink in true_edges]
    return true_edges

In [ ]:
# Create arbitrary amount of fake pairs
def create_fake_edges(num_obs, graph_nodes, graph):
    k=0
    false_edges = []
    while(k < num_obs):
        #chose two random nodes
        node1, node2 = random.choices(graph_nodes, k=2)
        if not graph.has_edge(node1,node2):
            false_edges.append([node1,node2])
            k=k+1
    return false_edges

In [ ]:
#Prepare data for training and test
graph_edges = list(G.edges)
graph_nodes = list(G.nodes)
edges_real = create_real_edges(50000, graph_edges)
edges_fake = create_fake_edges(50000, graph_nodes, G)

In [ ]:
def save_list2csv(path, data, header):
    with open(path, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(data)

In [ ]:
#Save training edges in files
save_list2csv("function_real.csv", edges_real, ['Source', 'Sink'])
save_list2csv("function_fake.csv", edges_fake, ['Source', 'Sink'])

In [ ]:
len(edges_real[1:])

# 2. Load training edges from files and calculate features

In [ ]:
#returns a list of lists, all floats no headers
def read_csv2list(path):
    observations = []
    with open(path, newline='') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        header = True
        for row in csv_reader:
            int_row = []
            if header == False:
                for element in row:
                    int_row.append(float(element))
                observations.append(int_row)
            header = False
            
    return observations

In [ ]:
edges_real = read_csv2list('edges_real.csv')
edges_fake = read_csv2list('edges_fake.csv')

In [ ]:
def get_in_edges(graph, node):
    edges = [x for x, y in graph.in_edges(node)]
    return edges

def get_out_edges(graph, node):
    edges = [y for x, y in graph.out_edges(node)]
    return edges

def get_all_edges(graph, node):
    return list(set(get_in_edges(graph, node) + get_out_edges(graph, node)))
    

In [ ]:
def create_features(edges, graph):
    features = []
    for source, sink in edges:
        source =int(source)
        sink = int(sink)
        #common in-neighbours
        so_in_edges = get_in_edges(graph, source)
        si_in_edges = get_in_edges(graph, sink)
        in_common = [value for value in so_in_edges if value in si_in_edges]
        
        #common out-neighbours
        so_out_edges = get_out_edges(graph, source)
        si_out_edges = get_out_edges(graph, sink)
        out_common = [value for value in so_out_edges if value in si_out_edges]      
        
        #in-Jaccard Coefficient
        in_union = list(set().union(so_in_edges, si_in_edges))
        if len(in_union) == 0:
            in_JC = -1
        else:
            in_JC = len(in_common)/len(in_union)
        
        #out-Jaccard Coefficient
        out_union = list(set().union(so_out_edges, si_out_edges))
        if len(out_union) == 0:
            out_JC = -1
        else:
            out_JC = len(out_common)/len(out_union)
                                    
        #In-Sorensen Index
        if len(so_in_edges) + len(si_in_edges) == 0:
            in_SI = -1
        else:
            in_SI = len(in_common)/(len(so_in_edges) + len(si_in_edges))
            
        #Out-Sorensen Index
        if len(so_out_edges) + len(si_out_edges) == 0:
            out_SI = -1
        else:
            out_SI = len(out_common)/(len(so_out_edges) + len(si_out_edges))

        #In-Hub Promoted
        if min([len(so_in_edges), len(si_in_edges)]) == 0:
            in_HP = -1
        else:
            in_HP = len(in_common)/(min([len(so_in_edges), len(si_in_edges)]))
        
        #Out-Hub Promoted
        if min([len(so_out_edges), len(si_out_edges)]) == 0:
            out_HP = -1
        else:
            out_HP = len(out_common)/(min([len(so_out_edges), len(si_out_edges)]))
            
        #In-Hub Depressed
        if max([len(so_in_edges), len(si_in_edges)]) == 0:
            in_HD = -1
        else:
            in_HD = len(in_common)/(max([len(so_in_edges), len(si_in_edges)]))
            
        #Out-Hub Depressed
        if max([len(so_out_edges), len(si_out_edges)]) == 0:
            out_HD = -1
        else:
            out_HD = len(out_common)/(max([len(so_out_edges), len(si_out_edges)]))
   
        #In-Resource Allocation 
        in_RA = sum([0 if (len(get_in_edges(graph, node)) == 0) else (1/len(get_in_edges(graph, node))) \
                     for node in in_common])
        #Out-Resource Allocation
        out_RA = sum([0 if (len(get_out_edges(graph, node)) == 0) else (1/len(get_out_edges(graph, node))) \
                     for node in out_common])  
        
        features.append([len(in_common), len(out_common), 
                         in_JC, out_JC,
                         in_SI, out_SI, 
                         in_HP, out_HP, 
                         in_HD, out_HD,
                         in_RA, out_RA]) 

    return features

In [ ]:
# graph to test previous functions
G2 = nx.DiGraph()
G2.add_edge(0, 1)
G2.add_edge(0, 2)
G2.add_edge(1, 2)
G2.add_edge(1, 3)
G2.add_edge(3, 0)
G2.add_edge(2, 0)
G2.neighbors(1)
test_ft = create_features([[0,1],[1,3]], G2)
test_ft

## 2.1 Subsets to calculate features in sections

In [ ]:
edges_real = read_csv2list('edges_real.csv')
edges_fake = read_csv2list('edges_fake.csv')

In [ ]:
training_edges = edges_real + edges_fake

part1 = training_edges[1:34000]
part2  = training_edges[34000:50001] + training_edges[50002:67000]
part3 = training_edges[67000:10000]

_features = create_features([training_edges[99999]], G) #Fill with subset

# 3. Model training and Prediction

In [ ]:
#read subset 
part1_features = read_csv2list('edges_features_part1.csv')
part2_features = read_csv2list('edges_features_part2.csv')[1000:]
part3_features = read_csv2list('edges_features_part3.csv')

print('part1 ' + str(len(part1_features)))
print('part2 ' + str(len(part2_features)))
print('part3 ' + str(len(part3_features)))

total_training_features = part1_features + part2_features + part3_features
len(total_training_features)

In [ ]:
save_list2csv('all_features.csv', total_training_features, ['in_common', 'out_common',
                                                         'in_JC', 'out_JC','in_SI', 'out_SI',
                                                         'in_HP', 'out_HP', 'in_HD', 'out_HD',  
                                                         'in_RA', 'out_RA'])

In [ ]:
#Split data for validation
from sklearn.model_selection import train_test_split
X = total_training_features
y = [1]*len(edges_real) + [0]*(len(edges_fake)-1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
# train model
from sklearn.linear_model import LogisticRegression
from sklearn import svm

model_LR = LogisticRegression(random_state=0).fit(X_train, y_train)
print ( 'Logistic Regression Acc ' + str(model_LR.score(X_test, y_test)))

model_SVM = svm.SVC()
model_SVM.fit(X_train, y_train)
print ( 'Logistic SVM Acc ' + str(model_SVM.score(X_test, y_test)))